# 모델 예측 및 결과 저장

이 노트북은 훈련된 모델을 사용하여 train data와 test data에 대한 예측을 수행하고 결과를 CSV 파일로 저장합니다.

## 주요 기능
- 훈련된 모델 로드
- Train data와 Test data에 대한 예측 수행
- 예측 결과를 CSV 파일로 저장
- 예측 성능 분석 및 시각화


In [1]:
# Library Import
import os
import math
import warnings
from collections import Counter
import platform
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler
from tqdm.auto import tqdm
import wandb

# Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    set_seed
)


# Sklearn
from sklearn.metrics import accuracy_score, f1_score, classification_report

print("✅ 라이브러리 임포트 완료")


/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotat

✅ 라이브러리 임포트 완료


In [2]:
# 환경 설정
RANDOM_STATE = 42
set_seed(RANDOM_STATE)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"디바이스: {device}")

if torch.cuda.is_available():
    print(f"GPU 개수: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("⚠️  CUDA 사용 불가 - CPU로 추론 진행")

os.environ["TOKENIZERS_PARALLELISM"] = "false"

import datetime
now = datetime.datetime.now()
TIMESTAMP = now.strftime("%Y-%m-%d_%H-%M-%S")


디바이스: cuda
GPU 개수: 1
   GPU 0: Tesla V100-SXM2-32GB


In [ ]:
# 설정
MODEL_FOLDER = "./Final_Models/original"
MODEL_NAME = "/data/ephemeral/home/code/Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42_epoch_3"
MODEL_PATH = os.path.join(MODEL_FOLDER, MODEL_NAME)  # 훈련된 모델 경로

TRAIN_DATA_PATH = "/data/ephemeral/home/code/data/train_final_augX3.csv"
TEST_DATA_PATH = "/data/ephemeral/home/code/data/test_processed.csv"
VAL_DATA_PATH = "/data/ephemeral/home/code/data/val_final.csv"

# 출력 경로
OUTPUT_DIR = "./predictions"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 클래스 라벨 매핑
LABEL_MAPPING = {0: "매우 부정", 1: "부정", 2: "긍정", 3: "매우 긍정"}
REVERSE_LABEL_MAPPING = {v: k for k, v in LABEL_MAPPING.items()}

print("✅ 설정 완료")
print(f"모델 경로: {MODEL_PATH}")
print(f"출력 디렉토리: {OUTPUT_DIR}")


✅ 설정 완료
모델 경로: /data/ephemeral/home/code/Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_07-21-18_RANDOM_42_epoch_2
출력 디렉토리: ./predictions


In [4]:
PROJECT_NAME = f"[domain_project]_Ensemble_Models"

RUN_NAME = f"{MODEL_NAME}"

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
# 데이터셋 클래스 정의 (finetuning_pytorch.ipynb와 동일)
class ReviewDataset(Dataset):
    """
    리뷰 텍스트 데이셋 클래스
    - BERT 모델 훈련/추론을 위한 PyTorch Dataset 구현
    - 텍스트 토크나이징 및 텐서 변환 처리
    """

    def __init__(self, texts, labels, tokenizer, max_length):
        """
        데이터셋 초기화
        """
        self.texts, self.labels, self.tokenizer, self.max_length = (
            texts,
            labels,
            tokenizer,
            max_length,
        )

    def __len__(self):
        """데이터셋 크기 반환"""
        return len(self.texts)

    def __getitem__(self, idx):
        """
        특정 인덱스의 데이터 아이템 반환
        """
        # 텍스트 토크나이징 및 패딩
        encoding = self.tokenizer(
            str(self.texts.iloc[idx]),
            truncation=True,  # 최대 길이 초과시 자르기
            padding="max_length",  # 최대 길이까지 패딩
            max_length=self.max_length,
            return_tensors="pt",  # PyTorch 텐서로 반환
        )

        # 기본 아이템 구성 (input_ids, attention_mask)
        item = {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
        }

        # labels가 None이 아닌 경우에만 추가 (train/valid용)
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)

        return item

print("✅ 데이터셋 클래스 정의 완료")


✅ 데이터셋 클래스 정의 완료


In [6]:
# 모델 로드 함수
def load_model(model_path):
    """훈련된 모델과 토크나이저를 로드하는 함수"""
    print(f"모델 로드 중: {model_path}")
    
    if not os.path.exists(model_path):
        print(f"❌ 모델 경로가 존재하지 않습니다: {model_path}")
        return None, None
    
    try:
        # 토크나이저 로드
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        print("✅ 토크나이저 로드 완료")
        
        # 모델 로드
        model = AutoModelForSequenceClassification.from_pretrained(
            model_path,
            num_labels=4,
            ignore_mismatched_sizes=True
        )
        model = model.to(device)
        model.eval()
        print("✅ 모델 로드 완료")
        
        return model, tokenizer
        
    except Exception as e:
        print(f"❌ 모델 로드 실패: {str(e)}")
        return None, None

# 모델 로드
model, tokenizer = load_model(MODEL_PATH)

if model is None or tokenizer is None:
    print("❌ 모델 로드에 실패했습니다. 프로그램을 종료합니다.")
    exit()


모델 로드 중: /data/ephemeral/home/code/Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_07-21-18_RANDOM_42_epoch_2
✅ 토크나이저 로드 완료
✅ 모델 로드 완료


In [7]:
# 예측 함수
def predict_dataset(model, tokenizer, dataloader, device, dataset_name="Dataset"):
    """데이터셋에 대한 예측을 수행하는 함수"""
    print(f"\\n📊 {dataset_name} 예측 시작...")
    
    model.eval()
    all_predictions = []
    all_probabilities = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Predicting {dataset_name}"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            
            # 예측 수행
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            
            # 확률 계산
            probabilities = torch.softmax(logits, dim=1)
            
            # 예측 클래스
            predictions = torch.argmax(logits, dim=1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
            
            # 라벨이 있는 경우 (train/val 데이터)
            if "labels" in batch:
                all_labels.extend(batch["labels"].cpu().numpy())
    
    print(f"✅ {dataset_name} 예측 완료: {len(all_predictions):,}개")
    
    return {
        "predictions": np.array(all_predictions),
        "probabilities": np.array(all_probabilities),
        "labels": np.array(all_labels) if all_labels else None
    }

print("✅ 예측 함수 정의 완료")


✅ 예측 함수 정의 완료


In [8]:
# 데이터 로드
print("📂 데이터 로드 중...")

# Train 데이터 로드
train_df = pd.read_csv(TRAIN_DATA_PATH)
train_df = train_df[train_df["type"] == "original"]
print(f"Train 데이터: {len(train_df):,}개")

# Test 데이터 로드
test_df = pd.read_csv(TEST_DATA_PATH)
print(f"Test 데이터: {len(test_df):,}개")

# Val 데이터 로드 (성능 평가용)
val_df = pd.read_csv(VAL_DATA_PATH)
print(f"Val 데이터: {len(val_df):,}개")

print("✅ 데이터 로드 완료")


📂 데이터 로드 중...
Train 데이터: 129,630개
Test 데이터: 59,928개
Val 데이터: 6,823개
✅ 데이터 로드 완료


In [9]:
# 데이터셋 및 데이터로더 생성
print("🔧 데이터셋 및 데이터로더 생성 중...")

# 설정
MAX_LENGTH = 128
BATCH_SIZE = 2048

# Train 데이터셋
train_dataset = ReviewDataset(
    train_df["review"],
    train_df["label"],
    tokenizer,
    MAX_LENGTH
)

# Test 데이터셋
test_dataset = ReviewDataset(
    test_df["review"],
    None,  # Test 데이터는 라벨 없음
    tokenizer,
    MAX_LENGTH
)

# Val 데이터셋
val_dataset = ReviewDataset(
    val_df["review"],
    val_df["label"],
    tokenizer,
    MAX_LENGTH
)

# 데이터로더 생성
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

print("✅ 데이터셋 및 데이터로더 생성 완료")


🔧 데이터셋 및 데이터로더 생성 중...
✅ 데이터셋 및 데이터로더 생성 완료


In [10]:
# 예측 수행
print("🚀 예측 수행 시작...")

# Train 데이터 예측
#train_results = predict_dataset(model, tokenizer, train_dataloader, device, "Train")

# Test 데이터 예측
test_results = predict_dataset(model, tokenizer, test_dataloader, device, "Test")

# Val 데이터 예측 (성능 평가용)
val_results = predict_dataset(model, tokenizer, val_dataloader, device, "Validation")

print("✅ 모든 예측 완료!")


🚀 예측 수행 시작...
\n📊 Test 예측 시작...


Predicting Test:   0%|          | 0/30 [00:00<?, ?it/s]

✅ Test 예측 완료: 59,928개
\n📊 Validation 예측 시작...


Predicting Validation:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Validation 예측 완료: 6,823개
✅ 모든 예측 완료!


In [11]:
# # 성능 평가 (Train, Val 데이터)
# def evaluate_predictions(predictions, labels, dataset_name):
#     """예측 결과를 평가하는 함수"""
#     print(f"\\n📊 {dataset_name} 성능 평가:")
    
#     accuracy = accuracy_score(labels, predictions)
#     f1 = f1_score(labels, predictions, average="weighted")
    
#     print(f"  정확도: {accuracy:.4f}")
#     print(f"  F1 점수: {f1:.4f}")
    
#     # 클래스별 성능
#     print(f"\\n  클래스별 성능:")
#     report = classification_report(labels, predictions, 
#                                  target_names=list(LABEL_MAPPING.values()),
#                                  output_dict=True)
    
#     for i, class_name in LABEL_MAPPING.items():
#         if str(i) in report:
#             precision = report[str(i)]["precision"]
#             recall = report[str(i)]["recall"]
#             f1_score_class = report[str(i)]["f1-score"]
#             support = report[str(i)]["support"]
#             print(f"    {class_name}: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1_score_class:.3f}, Support={support}")
    
#     return accuracy, f1

# # Train 데이터 성능 평가
# train_accuracy, train_f1 = evaluate_predictions(
#     train_results["predictions"], 
#     train_results["labels"], 
#     "Train"
# )

# # Val 데이터 성능 평가
# val_accuracy, val_f1 = evaluate_predictions(
#     val_results["predictions"], 
#     val_results["labels"], 
#     "Validation"
# )


In [12]:
# 예측 결과를 DataFrame에 추가
print("\\n📝 예측 결과를 DataFrame에 추가 중...")

# # Train 데이터에 예측 결과 추가
# train_df["predicted_label"] = train_results["predictions"]
# train_df["predicted_class"] = train_df["predicted_label"]

# # 각 클래스별 확률 추가
# for i in range(4):
#     train_df[f"prob_class_{i}"] = train_results["probabilities"][:, i]

# Test 데이터에 예측 결과 추가
test_df["pred"] = test_results["predictions"]


# 각 클래스별 확률 추가
for i in range(4):
    test_df[f"prob_class_{i}"] = test_results["probabilities"][:, i]

# Val 데이터에 예측 결과 추가
# val_df["pred"] = val_results["predictions"]


# # 각 클래스별 확률 추가
# for i in range(4):
#     val_df[f"prob_class_{i}"] = val_results["probabilities"][:, i]

print("✅ 예측 결과 추가 완료")


\n📝 예측 결과를 DataFrame에 추가 중...
✅ 예측 결과 추가 완료


In [13]:
# CSV 파일 저장
print("\\n💾 CSV 파일 저장 중...")

# Train 데이터 저장
# train_output_path = os.path.join(OUTPUT_DIR, f"{MODEL_NAME}_train_predictions.csv")
# train_df.to_csv(train_output_path, index=False, encoding="utf-8")
# print(f"✅ Train 예측 결과 저장: {train_output_path}")

# Test 데이터 저장
test_output_path = os.path.join(OUTPUT_DIR, f"{MODEL_NAME}_test_predictions.csv")
test_df.to_csv(test_output_path, index=False, encoding="utf-8")
print(f"✅ Test 예측 결과 저장: {test_output_path}")

# # Val 데이터 저장
# val_output_path = os.path.join(OUTPUT_DIR, f"{MODEL_NAME}_val_predictions.csv")
# val_df.to_csv(val_output_path, index=False, encoding="utf-8")
# print(f"✅ Val 예측 결과 저장: {val_output_path}")

# 요약 통계 저장
# summary_stats = {
#     "dataset": ["Train", "Validation"],
#     "accuracy": [train_accuracy, val_accuracy],
#     "f1_score": [train_f1, val_f1],
#     "total_samples": [len(train_df), len(val_df)]
# }

# summary_df = pd.DataFrame(summary_stats)
# summary_path = os.path.join(OUTPUT_DIR, "prediction_summary.csv")
# summary_df.to_csv(summary_path, index=False, encoding="utf-8")
# print(f"✅ 요약 통계 저장: {summary_path}")

# print("\\n🎉 모든 파일 저장 완료!")


\n💾 CSV 파일 저장 중...


✅ Test 예측 결과 저장: /data/ephemeral/home/code/Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_07-21-18_RANDOM_42_epoch_2_test_predictions.csv
